In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
libs = {
    "fastapi": "https://context7.com/tiangolo/fastapi",
    "langchain": "https://context7.com/context7/python_langchain_com-docs-introduction",
    "pydantic": "https://context7.com/context7/pydantic_dev",
    "numpy": "https://context7.com/numpy/numpy",
    "pandas": "https://context7.com/pandas-dev/pandas",
    "requests": "https://context7.com/psf/requests",
    "flask": "https://context7.com/pallets/flask",
    "django": "https://context7.com/django/django",
    "matplotlib": "https://context7.com/matplotlib/matplotlib"
}

private_libs = [
    "alis_lib"
]

lib_names = list(libs.keys())
lib_names

['fastapi',
 'langchain',
 'pydantic',
 'numpy',
 'pandas',
 'requests',
 'flask',
 'django',
 'matplotlib']

In [11]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o", model_provider="openai")

# llm = init_chat_model("gpt-4.1-nano", model_provider="openai")

In [16]:
import importlib
import structured_outputs
from structured_outputs import Lib

importlib.reload(structured_outputs)

lib_extractor_llm = llm.with_structured_output(Lib)

In [19]:
code = """
import project_demo
import numpy

use project demo to calculate the dot product of 2 vectors
"""

ls = lib_extractor_llm.invoke(code)

print("Private repos: ")
print(ls.to_dict_private())

print("public repos: ")
print(ls.to_dict_public())

Private repos: 
{'project_demo': ' calculate dot product of two vectors'}
public repos: 
{'numpy': ' perform vector and matrix calculations'}


In [ ]:
# from pydantic import BaseModel, Field
# from typing import List

# class Lib(BaseModel):
#     f"""
#         You will receive a user prompt that may or may not contain code. Your task is to:
#         If the prompt contains code:
#         Analyze the code to extract any function calls or relevant usage patterns.
#         Identify which Python libraries are likely being used or would be helpful to solve the problem.
#         If the prompt contains only natural language (no code):
#         Infer what Python libraries might help solve the user's task, based on keywords or task intent (e.g., web scraping, data analysis, NLP, etc.).
#         Return a list of all relevant library names. Do not return duplicates.
#         The first word should in each entry should be the name of the library, and then there should be a small sentence 7 to 15 wrods at max indicating what are the things that i need from this library,
#         avoid using questions, if you need more that one thing insert this library with difference search sentence, i will search a vector database so write a seach sentence that workd best with vector databases
#         If the prompt is not code-related and has no technical request, return an empty list.
#         Suggest all libraries that are present in the code, if code is available
        
#         Constraints:
#         Don't the same library twice in the code unless its really needed for a different thing, not a thing similar to a previous topic
#         Be accurate — only suggest libraries that are truly applicable.
#         Do not hallucinate libraries unless their usage is clearly implied.
#         Include libraries even if the user doesn't mention them by name, but their functionality is required.
#     """

#     lib: List[str] = Field(
#         description=(
#             "List of identified libraries from the user prompt or history. "
#             "Each item must begin with the library name followed by a clear, concise task expressed as a direct command. "
#             "Use action verbs (e.g., 'build', 'visualize', 'compute') to start each task description — this improves relevance in vector search. "
#             "Avoid vague or filler phrases like 'implement endpoint for numerical computations with advanced math and visualization'. "
#             "Be specific about what you want the library to do. "
#             "No clutter — treat each description as if querying a vector database. "
#             "Always separate concerns, don't use FastAPI  .... for numerical calculations (ommit thes ething only focus on the purpose of the library) VERY IMPORTANT." \
#             "Always use the following formating `lib_name: seach sentence`."
#         )
#     )

#     def to_dict(self) -> dict:
        
#         out = {}
#         for item in self.lib:
#             name, search = item.split(':')
#             out[name.lower()] = search

#         filtered = {k: v for k, v in out.items() if k in lib_names}

#         return filtered

# class CodeTextSep(BaseModel):
#     """
#         You will be given a prompt that contains a mix of normal text and code.
#         Your task is not to analyze, modify, or interpret either part.
#         Simply separate the input into two distinct parts:
#         Text Section: This includes any explanatory text, instructions, or natural language written by the user.
#         Code Section: This includes only the code, exactly as written. Do not alter, reformat, or fix the code in any way.
#     """

#     text: str = Field(description="This is the text part of the promt")
#     code: str = Field(description="This is the code part of the promt")

# lib_extractor_llm = llm.with_structured_output(Lib)



In [13]:
code = """
import alis_lib

using alis_lib create a new finantial document
"""

lib_extractor_llm.invoke(code)

Lib(lib=['alis_lib: Create a new financial document.'])

In [25]:
query = """

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

tools_registry = {
    "scrap_docs": scrap_docs
}

tools = [scrap_docs]
llm_with_tools = llm.bind_tools(tools)

query = "Write a function that chains 2 llms using langchain"
messages = [
    SystemMessage("you are a very funny and helpful assistant, joke with the user and put silly funny comments in the code"),
    HumanMessage(query),
]

ai_message = llm_with_tools.invoke(query)

messages.append(ai_message)

tool_calls = ai_message.tool_calls

for tool_call in tool_calls:
    selected_tool = tools_registry[tool_call['name']]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

final_output = llm_with_tools.invoke(messages)

the above is the given code, i want it to be in a way that is reusable in code using things in langchain, i also want to do api requests

"""

lib_ext_output = lib_extractor_llm.invoke(query)
lib_dict = lib_ext_output.to_dict()

lib_dict

{'requests': ' make API requests to external services'}

In [26]:
from langchain_core.tools import tool

@tool
def scrap_docs(lib_name: str, topic: str) -> str:
    """
        Uses get_docs function to scrap documentation from context7, if code is provided
        use this code to reason about what the user want, the topic should be something related to the code
        and the prompt provided by the user, if there is code try reason what is the library used to scrap
        its docs using the get_docs function be more descriptive with the topic focus on making it relavant to what the user is promting you.
        You can use this tool multiple times.

        Args:
            lib_name: name of the library that we want to scrap that is provided by context7
            topic: topic of interets in the library to get more accurate and useful docs for out use case
    """

    return get_docs(libs[lib_name], topic)

In [14]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("Improve the following code and add descriptive comments:\n\n{code}")

prompt = (
    PromptTemplate.from_template("Improve the following code and add descriptive comments:\n\n{code}") + 
    ", make it funny" +
    ", make it {another_feeling}."
)

prompt

PromptTemplate(input_variables=['another_feeling', 'code'], input_types={}, partial_variables={}, template='Improve the following code and add descriptive comments:\n\n{code}, make it funny, make it {another_feeling}.')

In [27]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

def execute_llm(llm, query: str, system_promt: str):

    tools_registry = {
        "scrap_docs": scrap_docs
    }

    tools = [scrap_docs]
    llm_with_tools = llm.bind_tools(tools)

    messages = [
        SystemMessage(system_promt),
        HumanMessage(query),
    ]

    useful_libs = lib_extractor_llm.invoke(query)
    useful_libs = useful_libs.to_dict()

    libs_text = ""

    for k, v in useful_libs.items():
        libs_text += f"{k} search for `{v}`\n"

    next_prompt = f"get the docs and search for the topics of the following libraries\n{libs_text}"

    ai_message = llm_with_tools.invoke(next_prompt)
    messages.append(ai_message)

    tool_calls = ai_message.tool_calls

    for tool_call in tool_calls:
        selected_tool = tools_registry[tool_call['name']]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

    output = llm_with_tools.invoke(messages)
    return (output, messages)

system_prompt = "You are a very helpful AI coding assistance that write code from the prompt given, you focus on code everything else don't give it attention"
query = """

    from langchain.chat_models import init_chat_model

    llm = init_chat_model("gpt-4.1", model_provider="openai")

    def execute_llm(llm, query: str, system_promt: str):

        tools_registry = {
            "scrap_docs": scrap_docs
        }

        tools = [scrap_docs]
        llm_with_tools = llm.bind_tools(tools)

        messages = [
            SystemMessage(system_promt),
            HumanMessage(query),
        ]

        useful_libs = lib_extractor_llm.invoke(query)
        useful_libs = useful_libs.to_dict()

        libs_text = ""

        for k, v in useful_libs.items():
            libs_text += f"{k} search for `{v}`\n"

        next_prompt = f"get the docs and search for the topics of the following libraries\n{libs_text}"

        ai_message = llm_with_tools.invoke(next_prompt)
        messages.append(ai_message)

        tool_calls = ai_message.tool_calls

        for tool_call in tool_calls:
            selected_tool = tools_registry[tool_call['name']]
            tool_msg = selected_tool.invoke(tool_call)
            messages.append(tool_msg)

        output = llm_with_tools.invoke(messages)
        return (output, messages)

    create a simple fastapi app and wrap this code with a post request so that i can send a query and a system prompt from postman 
    
"""

output, messages = execute_llm(llm, query, system_prompt)

In [28]:
messages

[SystemMessage(content="You are a very helpful AI coding assistance that write code from the prompt given, you focus on code everything else don't give it attention", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n\n    from langchain.chat_models import init_chat_model\n\n    llm = init_chat_model("gpt-4.1", model_provider="openai")\n\n    def execute_llm(llm, query: str, system_promt: str):\n\n        tools_registry = {\n            "scrap_docs": scrap_docs\n        }\n\n        tools = [scrap_docs]\n        llm_with_tools = llm.bind_tools(tools)\n\n        messages = [\n            SystemMessage(system_promt),\n            HumanMessage(query),\n        ]\n\n        useful_libs = lib_extractor_llm.invoke(query)\n        useful_libs = useful_libs.to_dict()\n\n        libs_text = ""\n\n        for k, v in useful_libs.items():\n            libs_text += f"{k} search for `{v}`\n"\n\n        next_prompt = f"get the docs and search for the topics of the following libra

In [29]:
print(output.content)

Here's a FastAPI application that wraps the provided code with a POST endpoint for executing LLM queries and system prompts. This will allow you to send requests from Postman.

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage
from some_module import scrap_docs  # make sure to import your scrap_docs function correctly

# Initialize the chat model
llm = init_chat_model("gpt-4.1", model_provider="openai")

app = FastAPI()

# Define request model
class QueryRequest(BaseModel):
    query: str
    system_prompt: str

@app.post("/execute")
async def execute_query(request: QueryRequest):
    query = request.query
    system_prompt = request.system_prompt

    try:
        def execute_llm(llm, query: str, system_promt: str):
            tools_registry = {
                "scrap_docs": scrap_docs
            }
    
            tools = [scrap_docs]
 

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage
from helpers import scrap_docs  # make sure to import your scrap_docs function correctly

# Initialize the chat model
llm = init_chat_model("gpt-4.1", model_provider="openai")

app = FastAPI()

# Define request model
class QueryRequest(BaseModel):
    query: str
    system_prompt: str

@app.post("/execute")
async def execute_query(request: QueryRequest):
    query = request.query
    system_prompt = request.system_prompt

    try:
        def execute_llm(llm, query: str, system_promt: str):
            tools_registry = {
                "scrap_docs": scrap_docs
            }
    
            tools = [scrap_docs]
            llm_with_tools = llm.bind_tools(tools)
    
            messages = [
                SystemMessage(system_promt),
                HumanMessage(query),
            ]
    
            useful_libs = lib_extractor_llm.invoke(query)
            useful_libs = useful_libs.to_dict()
    
            libs_text = ""
    
            for k, v in useful_libs.items():
                libs_text += f"{k} search for `{v}`\n"
    
            next_prompt = f"get the docs and search for the topics of the following libraries\n{libs_text}"
    
            ai_message = llm_with_tools.invoke(next_prompt)
            messages.append(ai_message)
    
            tool_calls = ai_message.tool_calls
    
            for tool_call in tool_calls:
                selected_tool = tools_registry[tool_call['name']]
                tool_msg = selected_tool.invoke(tool_call)
                messages.append(tool_msg)
    
            output = llm_with_tools.invoke(messages)
            return (output, messages)
        
        result, full_messages = execute_llm(llm, query, system_prompt)
        return {"result": result, "full_messages": full_messages}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Add this line if you need to run the server using command: uvicorn <filename>:app --reload
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)